# Google+ Network

In [3]:
# # Install Necessary Packages
# install.packages("igraph")
# install.packages("Matrix")
# install.packages("pracma")
# install.packages("entropy")
# install.packages('infotheo')

# Load Packages 
library('igraph')
library('Matrix')
library('pracma')
library('igraph')
library('entropy')
library('infotheo')

**QUESTION 18:** Create directed personal networks for users who have more than 2 circles. The data required to create such personal networks can be found in the file named gplus.tar.gz. How many personal networks are there?

In [10]:
all_files = list.files(path="data/gplus", pattern="*.circles")
count = 0
for (file in all_files){
  circle_file = paste("data/gplus/", file, sep="")
  if (length(readLines(circle_file)) > 2){
    count = count + 1
  }
}
sprintf("Number of Personalized Networks with more than 2 circles is %d",count)

[1] "Number of Personalized Networks with more than 2 circles is 57"

**QUESTION 19:** For the 3 personal networks (node ID given below), plot the in-degree and outdegree distribution of these personal networks. Do the personal networks have a similar in and out degree distribution? In this question, you should have 6 plots.

• 109327480479767108490

• 115625564993990145546

• 101373961279443806744

In [31]:
# list of nodes.
nodes_list = c("109327480479767108490","115625564993990145546","101373961279443806744")
for (node in nodes_list){
    print(sprintf("Node ID: %s",node))
    node_file = paste("data/gplus/", node, ".edges", sep = "")
    node_graph = read_graph(node_file, format="ncol", directed=TRUE)
    node_graph = add.vertices(node_graph, nv = 1, name = node)
    
    
    index = which(V(node_graph)$name==node) 
  
    el = c()
    for (vertex in 1:(vcount(node_graph) - 1)) {
        el = c(el, c(index, vertex))
    }
    node_graph = add_edges(node_graph, el)
    
    
    print(sprintf("For the in-degree : mean=%5.3f, Variance=%5.3f", mean(degree(node_graph,mode="in")), var(degree(node_graph,mode="in"))))
    pdf(sprintf("plots/part2/q19_%s_in_degree.pdf",node))
    hist(degree(node_graph,mode="in"),main= paste("In-degree Distribution of Node ID ",node),xlab="Degree")
    dev.off()
    
    print(sprintf("For the out-degree : mean=%5.3f, Variance=%5.3f", mean(degree(node_graph,mode="out")), var(degree(node_graph,mode="out"))))
    pdf(sprintf("plots/part2/q19_%s_out_degree.pdf",node))
    hist(degree(node_graph,mode="out"),main= paste("Out-degree Distribution of Node ID ",node),xlab="Degree")
    dev.off()
}

[1] "Node ID: 109327480479767108490"
[1] "For the in-degree : mean=14.062, Variance=96.001"
[1] "For the out-degree : mean=14.062, Variance=4588.177"
[1] "Node ID: 115625564993990145546"
[1] "For the in-degree : mean=43.640, Variance=1020.621"
[1] "For the out-degree : mean=43.640, Variance=9351.303"
[1] "Node ID: 101373961279443806744"
[1] "For the in-degree : mean=298.118, Variance=86408.770"
[1] "For the out-degree : mean=298.118, Variance=166186.737"


**QUESTION 20:** For the 3 personal networks picked in Question 19, extract the community
structure of each personal network using Walktrap community detection algorithm. Report the
modularity scores and plot the communities using colors. Are the modularity scores similar? In this
question, you should have 3 plots.

In [32]:
# list of nodes.
nodes_list = c("109327480479767108490","115625564993990145546","101373961279443806744")
for (node in nodes_list){
    print(sprintf("Node ID: %s",node))
    node_file = paste("data/gplus/", node, ".edges", sep = "")
    node_graph = read_graph(node_file, format="ncol", directed=TRUE)
    node_graph = add.vertices(node_graph, nv = 1, name = node)
    
    
    index = which(V(node_graph)$name==node) 
  
    el = c()
    for (vertex in 1:(vcount(node_graph) - 1)) {
        el = c(el, c(index, vertex))
    }
    node_graph = add_edges(node_graph, el)
    
    node_community  = walktrap.community(node_graph)
    node_modularity = modularity(node_community)
    print(sprintf("Modularity of Node ID %s is %2.6f",node, node_modularity))
    

    pdf(sprintf("plots/part2/q20_%s_community_structure.pdf",node))
    plot(node_community, node_graph, main=sprintf("Community Structure (node ID =%s)",node),vertex.size=5, vertex.label=NA, edge.color = "grey", layout=layout.fruchterman.reingold)
    dev.off()
    

}

[1] "Node ID: 109327480479767108490"
[1] "Modularity of Node ID 109327480479767108490 is 0.252765"
[1] "Node ID: 115625564993990145546"
[1] "Modularity of Node ID 115625564993990145546 is 0.319473"
[1] "Node ID: 101373961279443806744"
[1] "Modularity of Node ID 101373961279443806744 is 0.191090"


**QUESTION 21:** Based on the expression for h and c, explain the meaning of homogeneity and
completeness in words.

In [ ]:
# Answer in the report


**QUESTION 22:** Compute the h and c values for the community structures of the 3 personal
network (same nodes as Question 19). Interpret the values and provide a detailed explanation. Are
there negative values? Why?

In [59]:
# list of nodes.
nodes_list = c("109327480479767108490","115625564993990145546","101373961279443806744")
for (node in nodes_list){
    strrep('**',75)
    print(sprintf("Node ID: %s",node))
    node_circles_file_name = paste("data/gplus/",node, ".circles", sep="")
    node_circles_file_content = readLines(file(node_circles_file_name, open="r"))
    
    # Get the circles for this node ID. 
    circles = list()
    for (var in 1:length(node_circles_file_content)) {
        circle_nodes = strsplit(node_circles_file_content[var],"\t")
        circles = c(circles, list(circle_nodes[[1]][-1]))
    }
    
    # Calculate the 'N' term.
    all_circles=c()
    for ( circle in circles)
        all_circles=c(all_circles,circle)
    all_circle=unique(all_circles)
    N=length(all_circle)
    
    # Calculating H(C) term. 
    h_c =0
    for (circle in circles){
        a_i = length(circle)
        h_c=h_c-((a_i/N)* log10(a_i/N))
        
    }

    # Find the community structure
    node_edges_file = paste("data/gplus/", node, ".edges", sep = "")
    node_graph = read_graph(node_edges_file, format="ncol", directed=TRUE)
    node_graph = add.vertices(node_graph, nv = 1, name = node)
    index = which(V(node_graph)$name==node) 
    el = c()
    for (vertex in 1:(vcount(node_graph) - 1)) {
        el = c(el, c(index, vertex))
    }
    node_graph = add_edges(node_graph, el)
    node_community  = walktrap.community(node_graph)
    
    #Find Community related entropy terms H(K) and conditional terms H(C|K) and H(K|C). 
    
    h_ck=0
    h_kc=0
    h_k=0
    
    # check percentage of match for walktrap community 
    for(m in 1:max(node_community$membership)){

        community_nodes = V(node_graph)$name[which(node_community$membership == m)]
        b_i=length(intersect(community_nodes,all_circle))
        if(b_i!=0) {
            h_k<- h_k - ((b_i/N) * log10(b_i/N))
            for (n in 1:length(circles)) {
                common_nodes = intersect(community_nodes, circles[[n]])
                c_ij<-length(unique(common_nodes))
                A_i<-length(circles[[n]])
                if(c_ij!=0){
                    h_ck=h_ck- ((c_ij/N)*log10(c_ij/b_i))
                    h_kc = h_kc - ((c_ij/N)*log10(c_ij/A_i))
                }
            }
        }
    }
    
    # Calculate homogenity, completeness and V-measure. 
    
    homogenity =  1-(h_ck/h_c)
    completeness = 1- (h_kc/h_k)
    v_measure = 2*(homogenity*completeness)/(homogenity+completeness)
    
    print(sprintf("Entropy H(C) = %5.8f",h_c))
    print(sprintf("Entropy H(K) = %5.8f",h_k))
    print(sprintf("Conditional Entropy H(C|K) = %5.8f",h_ck))
    print(sprintf("Conditional Entropy H(K|C) = %5.8f",h_kc))
    
    print(sprintf("Homogeneity h= %5.8f", homogenity))
    print(sprintf("Completeness c= %5.8f", completeness))
    
    print(sprintf("V-measure V= %5.8f", v_measure))
    cat("\n")
}


[1] "Node ID: 109327480479767108490"
[1] "Entropy H(C) = 0.45634767"
[1] "Entropy H(K) = 0.43655637"
[1] "Conditional Entropy H(C|K) = 0.06759188"
[1] "Conditional Entropy H(K|C) = 0.29254781"
[1] "Homogeneity h= 0.85188512"
[1] "Completeness c= 0.32987391"
[1] "V-measure V= 0.47558710"

[1] "Node ID: 115625564993990145546"
[1] "Entropy H(C) = 3.67636649"
[1] "Entropy H(K) = 0.46955527"
[1] "Conditional Entropy H(C|K) = 2.01505212"
[1] "Conditional Entropy H(K|C) = 2.07729483"
[1] "Homogeneity h= 0.45189030"
[1] "Completeness c= -3.42396235"
[1] "V-measure V= 1.04119642"

[1] "Node ID: 101373961279443806744"
[1] "Entropy H(C) = 0.16690804"
[1] "Entropy H(K) = 0.21425076"
[1] "Conditional Entropy H(C|K) = 0.16626265"
[1] "Conditional Entropy H(K|C) = 0.53653499"
[1] "Homogeneity h= 0.00386671"
[1] "Completeness c= -1.50423839"
[1] "V-measure V= 0.00775334"

